## Import librairie :

In [1]:
import requests
#Packages
#--Web scraping packages
from bs4 import BeautifulSoup
import requests
#Pandas/numpy for data manipulation
import pandas as pd
import numpy as np
import time

# Commerce :

In [2]:
#récupère les données des services
chiffre = requests.get('https://www.bien-dans-ma-ville.fr/puy-en-velay-43157').text
soup = BeautifulSoup(chiffre, "html.parser")
services = soup.find("section",{"id":"services"})
all_tab_html=services.findAll("td")
dico_service={}
for i in all_tab_html :
    if (str(i.contents[0])=='<i class="fas fa-times"></i>'):
        dico_service[j]=0
    elif (i.contents[0].isdigit()) :
        dico_service[j]=int(i.contents[0])
    else :
        j=str(i.contents[0]).replace('\t','').replace('\n','')
    
df_service=pd.DataFrame(dico_service, index=[0])
df_service

,Hypermarché,Supermarché,Boulangerie,Boucherie,Station service,Banque,La poste,Coiffeur,Vétérinaire,Restaurant,Médecin,Dentiste,Pharmacie,Maternelle,Primaire,Collège,Lycée
0,0,2,23,23,3,20,2,52,5,133,413,28,18,13,13,5,7


# Notes moyennes :

In [3]:
#récupère les notes des avis
webpage = requests.get('https://www.bien-dans-ma-ville.fr/puy-en-velay-43157/avis.html').text
soupavis = BeautifulSoup(webpage, "html.parser")
intro = soupavis.find("section", {"id": "intro"})
notesHtml=intro.findAll("span",{"class":"compteur"})
dicoNotes={}
tabCategorie=["Securite", "Education","Loisir / Sport","Environnement","Pratique"]
j=0
for i in notesHtml :
    dicoNotes[tabCategorie[j]]=float(i.contents[0])
    j+=1
dfNote=pd.DataFrame(dicoNotes, index=[0])
dfNote

,Securite,Education,Loisir / Sport,Environnement,Pratique
0,3.7,3.7,4.0,4.2,3.8


# Grosse fonction de la mort XX :

In [4]:
def recupData(lien, insee):
  #Récupère les commerces
  chiffre = requests.get(lien).text
  soup = BeautifulSoup(chiffre, "html.parser")
  services = soup.find("section",{"id":"services"})
  all_tab_html=services.findAll("td")
  dico_service={}
  for i in all_tab_html :
      if (str(i.contents[0])=='<i class="fas fa-times"></i>'):
          dico_service[j]=0
      elif (i.contents[0].isdigit()) :
          dico_service[j]=int(i.contents[0])
      else :
          j=str(i.contents[0]).replace('\t','').replace('\n','')
      
  df_service=pd.DataFrame(dico_service, index=[0])

  #Récupère info Ville
  chiffre_web = soup.find_all('section', id='chiffres')

  Insee = insee
  Ville = []
  Code_postal = []
  Nb_habitants = []
  Superficie = []
  Pop_densite = []
  Pop_active = []
  Taux_chomage = []
  Revenu_moyen = []
  Prix_moyen = []
  Tranche_population_0_14 = []
  Activite_professionnelle = [] 
  Note_global = [] 

  for comp in chiffre_web:
      Ville.append(soup.find_all('h1')[0].text.strip()[:-6])
      Code_postal.append(soup.find_all('h1')[0].text.strip()[-5:])
      Nb_habitants.append(soup.find_all('td')[1].text.strip())
      Superficie.append(soup.find_all('td')[4].text.strip())
      Pop_densite.append(soup.find_all('td')[7].text.strip())
      Pop_active.append(soup.find_all('td')[10].text.strip())
      Taux_chomage.append(soup.find_all('td')[13].text.strip())
      Revenu_moyen.append(soup.find_all('td')[16].text.strip())
      Prix_moyen.append(soup.find_all('td')[19].text.strip())
      if(soup.find("canvas",id="chart_age")["data-data"] is not None):
        Tranche_population_0_14.append(soup.find("canvas",id="chart_age")["data-data"])
      else :
        Tranche_population_0_14.append(np.nan)
      Activite_professionnelle.append(soup.find("canvas",id="chart_metier")["data-data"])
      if (soup.find("div", {"class": "total compteur"}) is not None ):
        Note_global.append(soup.find("div", {"class": "total compteur"}).contents[0])
      else :
        Note_global.append(np.nan)
  #creating dataframe for all list
  features = {'Insee':Insee, 'Code_postal':Code_postal, 'Ville':Ville,  'Nb_habitants':Nb_habitants, 'Superficie':Superficie, 'Pop_densite':Pop_densite,
            'Pop_active':Pop_active, 'Taux_chomage':Taux_chomage, 'Revenu_moyen':Revenu_moyen, 'Prix_moyen':Prix_moyen,
            'Tranche_population_0_14':Tranche_population_0_14, 'Activite_professionnelle':Activite_professionnelle,
            'Note_global':Note_global}
  df = pd.DataFrame(features)

  #Creation d'un nouveau dataframe pour les tranches de populations
  dfTranche_population=df['Tranche_population_0_14']
  dfTranche_population = dfTranche_population.str[1:-1]
  TotListeAge = dfTranche_population.str.split(",").apply(pd.Series)
  TotListeAge
  listeAge = ['0-14 ans','15-29 ans', '30-44 ans', '45-59 ans','60-74 ans','75-89 ans','90+ ans']
  dfListeAgeNom=pd.DataFrame(listeAge)
  dfListeAgeNom=dfListeAgeNom.transpose()
  dfListeAge=pd.concat([dfListeAgeNom, TotListeAge])
  headers = dfListeAge.iloc[0]
  dfListeAge  = pd.DataFrame(dfListeAge.values[1:], columns=headers)
  dfListeAge

  #Creation d'un nouveau dataframe pour les activités professionelle
  dfActivite_professionnelle=df['Activite_professionnelle']
  dfActivite_professionnelle = dfActivite_professionnelle.str[1:-1]
  TotActivites = dfActivite_professionnelle.str.split(",").apply(pd.Series)
  TotActivites
  listeActivites = ['Agriculteurs','Artisants/Commercants', 'Cadres', 'Profession intermédiaire','Employes','Ouvriers','Retraites','Sans emploie']
  dfActivitesNom=pd.DataFrame(listeActivites)
  dfActivitesNom=dfActivitesNom.transpose()
  dfListeActivites=pd.concat([dfActivitesNom, TotActivites])
  headersActivites = dfListeActivites.iloc[0]
  dfListeActivites  = pd.DataFrame(dfListeActivites.values[1:], columns=headersActivites)
  dfListeActivites

  #récupère les notes des avis
  webpage = requests.get(lien+'/avis.html').text
  soupavis = BeautifulSoup(webpage, "html.parser")
  intro = soupavis.find("section", {"id": "intro"})
  notesHtml=intro.findAll("span",{"class":"compteur"})
  dicoNotes={}
  tabCategorie=["Securite", "Education","Loisir / Sport","Environnement","Pratique"]
  j=0
  for i in notesHtml :
      dicoNotes[tabCategorie[j]]=float(i.contents[0])
      j+=1
  dfNote=pd.DataFrame(dicoNotes, index=[0])

  #Créer un dataframe des commentaire
  com = soupavis.findAll("div",{"class":"commentaire"})
  avis = []
  for i in com :
    dico_com={}
    dico_com['Insee']=df['Insee'][0]
    dico_com['Code_postal']=df['Code_postal'][0]
    dico_com['Ville']=df['Ville'][0]
    k=0
    for j in i.findAll("span"):  
      if(j.has_attr('title')) :
        dico_com[j['title']]=int(j.contents[0])
      elif (k==0 and not j.has_attr('class')):
        dico_com['pouce_positif']=int(j.contents[0])
        k+=1
      elif( not j.has_attr('class')):
        dico_com['pouce_negatif']=int(j.contents[0])
    dico_com['commentaire']=i.find('p',{'class':'description'}).contents[0]
    dico_com['date']=i.find('div',{'class':'date'}).contents[0]
    dico_com['auteur']=i.find('div',{'class':'auteur'}).contents[0]
    avis.append(dico_com)
  df_com=pd.DataFrame(avis)
  

  dfConcat=pd.concat([df, dfListeActivites,dfListeAge,dfNote,df_service], axis=1)
  dfConcat=dfConcat.drop(columns=['Tranche_population_0_14','Activite_professionnelle'],axis=1)
  return dfConcat, df_com

In [5]:
dfVille, dfComm = recupData('https://www.bien-dans-ma-ville.fr/soleilhas-04210','124455')
dfVille2, dfComm2 = recupData('https://www.bien-dans-ma-ville.fr/puy-en-velay-43157','32433')
dfConcat=pd.concat([dfVille,dfVille2], axis=0)
dfConcat

,Insee,Code_postal,Ville,Nb_habitants,Superficie,Pop_densite,Pop_active,Taux_chomage,Revenu_moyen,Prix_moyen,Note_global,Agriculteurs,Artisants/Commercants,Cadres,Profession intermédiaire,Employes,Ouvriers,Retraites,Sans emploie,0-14 ans,15-29 ans,30-44 ans,45-59 ans,60-74 ans,75-89 ans,90+ ans,Hypermarché,Supermarché,Boulangerie,Boucherie,Station service,Banque,La poste,Coiffeur,Vétérinaire,Restaurant,Médecin,Dentiste,Pharmacie,Maternelle,Primaire,Collège,Lycée,Securite,Education,Loisir / Sport,Environnement,Pratique
0,124455,04120,Soleilhas,120,35 km²,3 h/km²,45.8%,5.8%,NC,1 258 €,NaN,4,4,4,4,4,0,25,8,13,6,8,17,23,9,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
0,32433,43000,Le Puy-en-Velay,18 909,17 km²,1 112 h/km²,43.5%,8%,18 086 €/an,1 359 €,3.9,0,3,5,11,15,10,26,16,15,21,17,18,17,10,2,0,2,23,23,3,20,2,52,5,133,413,28,18,13,13,5,7,3.7,3.7,4.0,4.2,3.8


# Fichier (excel) bien dans ma ville

In [6]:
df = pd.read_csv('communes_url.csv',sep=';')
df.head()

,index,code,nom,dep,url
0,0,1001,abergement-clemenciat,1,https://www.bien-dans-ma-ville.fr/abergement-c...
1,1,1002,abergement-de-varey,1,https://www.bien-dans-ma-ville.fr/abergement-d...
2,2,1004,amberieu-en-bugey,1,https://www.bien-dans-ma-ville.fr/amberieu-en-...
3,3,1005,amberieux-en-dombes,1,https://www.bien-dans-ma-ville.fr/amberieux-en...
4,4,1006,ambleon,1,https://www.bien-dans-ma-ville.fr/ambleon-01-0...


In [7]:
url = df['url']
code = df['code']
index = df['index']

In [8]:
i=0
liste_ville = []
liste_comm = []

while i != 4000:
   dfVille , dfComm = recupData(url[i],code[i])
   liste_ville.append(dfVille)
   liste_comm.append(dfComm)
   i+=1

# time.sleep(50)

# while i != 8000:
#    dfVille , dfComm = recupData(url[i],code[i])
#    liste_ville.append(dfVille)
#    liste_comm.append(dfComm)
#    i+=1


ConnectionError: ignored

In [ ]:
columns=['Insee',	'Code_postal',	'Ville',	'Nb_habitants',	'Superficie',	'Pop_densite',	'Pop_active',	'Taux_chomage',	'Revenu_moyen',	'Prix_moyen',	'Note_global',	'Agriculteurs',	'Artisants/Commercants',	'Cadres',	'Profession', 'intermédiaire',	'Employes',	'Ouvriers',	'Retraites',	'Sans emploie',	'0-14 ans',	'15-29 ans',	'30-44 ans',	'45-59 ans',	'60-74 ans',	'75-89 ans',	'90+ ans',	'Hypermarché',	'Supermarché',	'Boulangerie',	'Boucherie'	,'Station service',	'Banque',	'La poste',	'Coiffeur',	'Vétérinaire',	'Restaurant',	'Médecin',	'Dentiste',	'Pharmacie',	'Maternelle',	'Primaire',	'Collège',	'Lycée',	'Securite',	'Education',	'Loisir / Sport',	'Environnement',	'Pratique']
df = pd.DataFrame(columns=columns)

col=['Insee',	'Code_postal',	'Ville',	'Sécurité',	'Éducation',	'Sport / Loisir',	'Environnement',	'Vie pratique',	'pouce_positif',	'pouce_negatif',	'commentaire',	'date',	'auteur']
df2 = pd.DataFrame(columns=col)

i=0
a=0

while i != len(liste_ville):
  df = df.append(liste_ville[i])
  i+=1


while a != len(liste_comm):
  df2 = df2.append(liste_comm[a])
  a+=1


In [ ]:
df.head()

In [ ]:
df2.head()

In [ ]:
df.to_excel(r'donneeSite.xlsx', index = False)
df2.to_excel(r'donneeCommentaire.xlsx', index = False)

# Information Ville/Population : 

In [ ]:
chiffre_web = soup.find_all('section', id='chiffres')

Ville = []
Nb_habitants = []
Superficie = []
Pop_densite = []
Pop_active = []
Taux_chomage = []
Revenu_moyen = []
Prix_moyen = []
Tranche_population_0_14 = []
Activite_professionnelle = [] 
Note_global = [] 

for comp in chiffre_web:
    display(soup.find_all('h1')[0].text.strip())
    Ville.append(soup.find_all('h1')[0].text.strip())
    Nb_habitants.append(soup.find_all('td')[1].text.strip())
    Superficie.append(soup.find_all('td')[4].text.strip())
    Pop_densite.append(soup.find_all('td')[7].text.strip())
    Pop_active.append(soup.find_all('td')[10].text.strip())
    Taux_chomage.append(soup.find_all('td')[13].text.strip())
    Revenu_moyen.append(soup.find_all('td')[16].text.strip())
    Prix_moyen.append(soup.find_all('td')[19].text.strip())
    Tranche_population_0_14.append(soup.find("canvas",id="chart_age")["data-data"])
    Activite_professionnelle.append(soup.find("canvas",id="chart_metier")["data-data"])
    Note_global.append(soup.find("div", {"class": "total compteur"}).contents[0])
#creating dataframe for all list
features = {'Ville':Ville, 'Nb_habitants':Nb_habitants, 'Superficie':Superficie, 'Pop_densite':Pop_densite,
           'Pop_active':Pop_active, 'Taux_chomage':Taux_chomage, 'Revenu_moyen':Revenu_moyen, 'Prix_moyen':Prix_moyen,
           'Tranche_population_0_14':Tranche_population_0_14, 'Activite_professionnelle':Activite_professionnelle,
           'Note_global':Note_global}
df = pd.DataFrame(features)
df

In [ ]:
dfVille, dfComm = recupData('https://www.bien-dans-ma-ville.fr/soleilhas-04210')
dfVille2, dfComm2 = recupData('https://www.bien-dans-ma-ville.fr/puy-en-velay-43157')
dfConcat=pd.concat([dfComm,dfComm2], axis=0)
dfConcat

# Tranche age population :

In [ ]:
#Creation d'un nouveau dataframe pour les tranches de populations
dfTranche_population=df['Tranche_population_0_14']
dfTranche_population = dfTranche_population.str[1:-1]
TotListeAge = dfTranche_population.str.split(",").apply(pd.Series)
TotListeAge
listeAge = ['0-14 ans','15-29 ans', '30-44 ans', '45-59 ans','60-74 ans','75-89 ans','90+ ans']
dfListeAgeNom=pd.DataFrame(listeAge)
dfListeAgeNom=dfListeAgeNom.transpose()
dfListeAge=pd.concat([dfListeAgeNom, TotListeAge])
headers = dfListeAge.iloc[0]
dfListeAge  = pd.DataFrame(dfListeAge.values[1:], columns=headers)
dfListeAge

# Tranche professionnel population :

In [ ]:
#Creation d'un nouveau dataframe pour les activités professionelle
dfActivite_professionnelle=df['Activite_professionnelle']
dfActivite_professionnelle = dfActivite_professionnelle.str[1:-1]
TotActivites = dfActivite_professionnelle.str.split(",").apply(pd.Series)
TotActivites
listeActivites = ['Agriculteurs','Artisants/Commercants', 'Cadres', 'Profession intermédiaire','Employes','Ouvriers','Retraites','Sans emploie']
dfActivitesNom=pd.DataFrame(listeActivites)
dfActivitesNom=dfActivitesNom.transpose()
dfListeActivites=pd.concat([dfActivitesNom, TotActivites])
headersActivites = dfListeActivites.iloc[0]
dfListeActivites  = pd.DataFrame(dfListeActivites.values[1:], columns=headersActivites)
dfListeActivites

# Fusion des données :

In [ ]:
#Concat entre le DF le dataframe des tranches de populations et le dataframe des activités professionelle
dfConcat=pd.concat([df, dfListeActivites,dfListeAge,dfNote,df_service], axis=1)
dfConcat

# Commentaires : 




In [ ]:
com = soupavis.findAll("div",{"class":"commentaire"})

avis = []
for i in com :
  dico_com={}
  dico_com['Ville']=df['Ville'][0]
  k=0
  for j in i.findAll("span"):  
    if(j.has_attr('title')) :
      dico_com[j['title']]=int(j.contents[0])
    elif (k==0 and not j.has_attr('class')):
      dico_com['pouce_positif']=int(j.contents[0])
      k+=1
    elif( not j.has_attr('class')):
      dico_com['pouce_negatif']=int(j.contents[0])
  dico_com['commentaire']=i.find('p',{'class':'description'}).contents[0]
  dico_com['date']=i.find('div',{'class':'date'}).contents[0]
  dico_com['auteur']=i.find('div',{'class':'auteur'}).contents[0]
  avis.append(dico_com)
df_com=pd.DataFrame(avis)
df_com